In [30]:
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd

In [31]:
data =  pd.read_csv('all_data_compiled_v2.csv')
print(data)

      index_compiled  index  class  B_M_FLUX_AP  B_B_FLUX_AP_AVG  \
0                  0    194    0.0     0.000222     0.000000e+00   
1                  1    464    0.0     0.000105     0.000000e+00   
2                  2   1496    1.0     0.000038     1.607810e-04   
3                  3    588    0.0     0.000011     0.000000e+00   
4                  4   1059    1.0     0.000005     8.777589e-07   
...              ...    ...    ...          ...              ...   
2558            2558   1418    1.0     0.000000     3.378608e-05   
2559            2559    456    0.0     0.000021     0.000000e+00   
2560            2560   1222    1.0     0.010319     2.393888e-02   
2561            2561    470    1.0     0.000008     2.410866e-06   
2562            2562    526    0.0     0.000016     1.448217e-06   

      B_B_FLUX_BB_AP  B_B_FLUX_BREMS_AP  B_B_FLUX_BREMS_AP90  \
0           0.000088           0.000074             0.000152   
1           0.000043           0.000037             0.0

In [32]:
print(data.shape)
x = data.drop(['index_compiled', 'index', 'class'],axis=1)
print(x.shape)
y = data['class']
y.shape

(2563, 117)
(2563, 114)


(2563,)

In [34]:
x_np =  x.to_numpy()
y_np =  y.to_numpy()
x_train = x_np[:1000,:]
y_train = y_np[:1000]
print(x_train.shape)
print(y_train.shape)

(1000, 114)
(1000,)


In [35]:
def split_data(x,y,s):   
    split = 0.7 
    split_no = int(split*len(y))
    x_train = x[:split_no, :]
    y_train = y[:split_no]
    x_test = x[split_no:, :]
    y_test = y[split_no:]
    return ((x_train,y_train) , (x_test ,y_test))

(x_train,y_train),(x_test,y_test) = split_data(x_np, y_np, 0.8)
print(x_train.shape ,y_train.shape)
print(x_test.shape ,y_test.shape)

(1794, 114) (1794,)
(769, 114) (769,)


# Neural Netork starts here

In [37]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [38]:
one_hot_y_train =  to_categorical(y_train)
one_hot_y_test =  to_categorical(y_test)

In [39]:
def model_gen(shape):

    inputs =  keras.Input(shape=(114,))
    dense =  layers.Dense(64, activation='relu')
    x = dense(inputs)
    x =  layers.BatchNormalization(axis=-1)(x)
    #x = layers.Dropout(0.3)(x)
    for s in shape:
        x = layers.Dense(64, activation='relu')(x)
    
    outputs = layers.Dense(2 , activation='sigmoid')(x)
    model = keras.Model(inputs=inputs , outputs=outputs , name='trial_model')
    model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(),
        metrics = ["accuracy"],
    )
    return model
model = model_gen([64,64,32,8])
history = model.fit(x_train, one_hot_y_train, batch_size=128, epochs=40, validation_split=0.1)

Epoch 1/40
13/13 [==============================] - 1s 19ms/step - loss: 0.5713 - accuracy: 0.6772 - val_loss: 0.6341 - val_accuracy: 0.7611
Epoch 2/40
13/13 [==============================] - 0s 4ms/step - loss: 0.4654 - accuracy: 0.7763 - val_loss: 0.6132 - val_accuracy: 0.7556
Epoch 3/40
13/13 [==============================] - 0s 5ms/step - loss: 0.4073 - accuracy: 0.8185 - val_loss: 0.6073 - val_accuracy: 0.7611
Epoch 4/40
13/13 [==============================] - 0s 6ms/step - loss: 0.3631 - accuracy: 0.8463 - val_loss: 0.5861 - val_accuracy: 0.7833
Epoch 5/40
13/13 [==============================] - 0s 5ms/step - loss: 0.3253 - accuracy: 0.8662 - val_loss: 0.5665 - val_accuracy: 0.7667
Epoch 6/40
13/13 [==============================] - 0s 5ms/step - loss: 0.2932 - accuracy: 0.8835 - val_loss: 0.5476 - val_accuracy: 0.7778
Epoch 7/40
13/13 [==============================] - 0s 5ms/step - loss: 0.2755 - accuracy: 0.8767 - val_loss: 0.5518 - val_accuracy: 0.7667
Epoch 8/40
13/13 [=

In [25]:
def acc_score(model , y_test , x_test):
    y_pred = model.predict(x_test)
    correct = 0
    total =  len(y_test)
    for y1,y2 in zip(y_pred,y_test):
        if(int(y1)==int(y2)):
            correct+=1
        else:
            continue
    print('correct prediction :' , correct)
    print('total prediction :' , total)
    print('score: ' , correct/total*100)
    
print('training data prediction')
acc_score(model, one_hot_y_train, x_train)
print('----------------------------------')
print('test data prediciton')
acc_score(model, y_test, x_test)

training data prediction


TypeError: only size-1 arrays can be converted to Python scalars

In [40]:
res = model.predict(x_test)
res = [np.argmax(r) for r in res]
print(res)

[1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 

In [41]:
count = 0
for a,b in zip(res,y_test):
    if(int(a)==int(b)):
        count+=1
print(count , len(y_test))
print(count/len(y_test))

667 769
0.8673602080624188
